## Тестирование модели и получение файла с прогнозом цены
####  Итоговый результат MAPE = 11.24149
Файл model_test.ipynb. Предназначен для предобработки данных тестового набора test.csv,
проверки на тестовом наборе 10 ранее обученных моделей и усреднения результатов.
В конце формируется файл submission_blend_best.csv, содержащий предсказанные цены на автомобили.

In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
import re
import json

## Подготовка тестового набора

In [4]:
DIR_TEST = 'data/'
test = pd.read_csv(DIR_TEST + 'test.csv')
sample_submission = pd.read_csv(DIR_TEST + 'sample_submission.csv')

In [6]:
test.head()

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,...,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
0,внедорожник 5 дв.,MERCEDES,синий,бензин,2015.0,250 2.0 AT (211 л.с.) 4WD,5.0,2017.0,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,...,50000.0,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,0
1,седан,INFINITI,чёрный,бензин,2014.0,3.7 AT (333 л.с.) 4WD,4.0,2014.0,SEDAN AUTOMATIC 3.7,автоматическая,...,78000.0,[],полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,1
2,хэтчбек 5 дв.,NISSAN,белый,бензин,2009.0,1.6 AT (110 л.с.),5.0,2013.0,HATCHBACK_5_DOORS AUTOMATIC 1.6,автоматическая,...,98000.0,"['[{""name"":""Элементы экстерьера"",""values"":[""Ст...",передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,2
3,внедорожник 5 дв.,BMW,серый,дизель,2013.0,M50d 3.0d AT (381 л.с.) 4WD,5.0,2016.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,...,69000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,3
4,внедорожник 5 дв.,MERCEDES,чёрный,бензин,2013.0,200 1.6 AMT (156 л.с.),5.0,2015.0,ALLROAD_5_DOORS ROBOT 1.6,роботизированная,...,116473.0,"['[{""name"":""Элементы экстерьера"",""values"":[""Ле...",передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,4


In [7]:
# Подгружаем заранее подготовленный список категорий для работы с признаком "комплектация"

with open('data/categories.list', 'r') as filehandle:
    categories = json.load(filehandle)
    
print(categories)

['Ксеноновые/Биксеноновые фары', 'Лазерные фары', 'Светодиодные фары', 'Электрообогрев зоны стеклоочистителей', 'Электрообогрев лобового стекла', 'Электрообогрев форсунок стеклоомывателей', 'Электрообогрев боковых зеркал', 'Противотуманные фары', 'Автоматический корректор фар', 'Омыватель фар', 'Система адаптивного освещения', 'Система управления дальним светом', 'Датчик дождя', 'Датчик света', 'Стальные диски', 'Легкосплавные диски', 'Диски 12', 'Диски 13', 'Диски 14', 'Диски 15', 'Диски 16', 'Диски 17', 'Диски 18', 'Диски 19', 'Диски 20', 'Диски 21', 'Диски 22', 'Диски 23', 'Диски 24', 'Диски 25', 'Диски 26', 'Диски 27', 'Диски 28', 'Аэрография', 'Обвес кузова', 'Рейлинги на крыше', 'Сигнализация', 'Сигнализация с обратной связью', 'Центральный замок', 'Иммобилайзер', 'Датчик проникновения в салон (датчик объема)', 'Аудиоподготовка', 'Аудиосистема', 'Аудиосистема Hi-Fi', 'Аудиосистема с TV', 'AUX', 'Bluetooth', 'USB', 'Мультимедиа система для задних пассажиров', 'Навигационная систем

In [10]:
def preproc_data_test(df_input):
    '''Предобработка тестового датасета'''
    
    df_output = df_input.copy()
    
    # ################### Предобработка ############################################################## 
    df_output.drop(['Таможня', 'Состояние', 'id','name','vehicleConfiguration','color','Руль', 'ПТС'], axis=1, inplace=True)
    df_output['Владельцы'] = df_output['Владельцы'].apply(new_owner)
    
    # Переводим признаки из float в int (иначе catboost выдает ошибку)
    for feature in ['modelDate', 'numberOfDoors', 'productionDate']:
        df_output[feature]=df_output[feature].astype('int32')
        
    # ################### Feature Engineering ####################################################
    df_output['mileage'] = df_output['mileage'].apply(define_mileage_category)
    df_output['enginePower'] = df_output['enginePower'].apply(lambda x: int(x.split(' ')[0]) )
    df_output['enginePower'] = df_output['enginePower'].apply(define_power_category)
    for item in categories:
        pattern = item.replace("(", "\(")
        pattern = pattern.replace(")", "\)")
        df_output[item] = df_output['Комплектация'].apply(lambda x: 1 if len(re.findall(pattern, x)) > 0 else 0)
    
    # ################### Clean #################################################### 
    df_output.drop(['Комплектация', 'description', 'Владение'], axis=1, inplace=True,)
    
    return df_output


def new_owner(owner):
    return owner.replace("\xa0", " ")

def define_mileage_category(mileage):
    mileage = mileage / 5000
    if mileage > 100:
        category = 101
    else:
        category = np.ceil(mileage).astype('int32')
    return category

def define_power_category(power):
    power = power / 25
    power = np.ceil(power).astype('int32')
    return power

In [11]:
X_sub = preproc_data_test(test)

In [12]:
# Подгружаем заранее подготовленный список лучших признаков
# из model.feature_importances_

with open('data/best_features.list', 'r') as filehandle:
    best_features = json.load(filehandle)

# Удаляем часть маловажных признаков
X_sub.drop(best_features[60:], axis=1, inplace=True)

## Объединение предсказаний моделей и усреднение результата

In [14]:
submissions = pd.DataFrame(0,columns=["sub_0"], index=sample_submission.index)

for i in range(5):
    from_file = CatBoostRegressor()
    model = from_file.load_model(f"models/catboost_8.5_{i}.model")
    submissions[f'sub_{i}'] = model.predict(X_sub)

for i in range(5):
    from_file = CatBoostRegressor()
    model = from_file.load_model(f"models/catboost_8.3_{i}.model")
    submissions[f'sub_8.3_{i}'] = model.predict(X_sub)
    
submissions.head(10)

,sub_0,sub_1,sub_2,sub_3,sub_4,sub_8.3_0,sub_8.3_1,sub_8.3_2,sub_8.3_3,sub_8.3_4
0,2.216409e+06,2.264539e+06,2.159175e+06,2.333180e+06,2.230331e+06,2.222726e+06,2.206875e+06,2.232819e+06,2.320340e+06,2.272892e+06
1,1.591311e+06,1.635465e+06,1.509524e+06,1.642508e+06,1.573701e+06,1.552598e+06,1.605340e+06,1.535138e+06,1.620572e+06,1.600218e+06
2,4.541008e+05,4.523012e+05,4.462238e+05,4.669259e+05,4.712850e+05,4.604676e+05,4.648495e+05,4.739556e+05,4.513737e+05,4.732883e+05
3,3.428869e+06,3.302874e+06,3.247518e+06,3.292152e+06,3.373475e+06,3.400247e+06,3.456046e+06,3.448713e+06,3.356943e+06,3.291047e+06
4,1.162176e+06,1.183500e+06,1.263492e+06,1.224902e+06,1.225088e+06,1.227423e+06,1.230519e+06,1.223758e+06,1.217109e+06,1.174262e+06
5,7.067415e+05,6.860136e+05,7.217564e+05,7.243815e+05,7.017758e+05,6.802832e+05,7.149302e+05,7.095903e+05,7.231357e+05,6.947482e+05
6,3.823382e+05,3.786617e+05,4.057125e+05,3.918948e+05,3.876224e+05,4.000389e+05,4.005123e+05,3.983157e+05,3.966572e+05,3.953623e+05
7,5.709626e+05,5.516947e+05,5.718534e+05,5.774139e+05,5.945654e+05,5.923105e+05,5.473129e+05,5.469589e+05,5.638524e+05,5.689892e+05
8,1.857120e+06,1.951354e+06,1.818032e+06,1.905901e+06,1.847699e+06,1.850514e+06,1.855425e+06,1.903565e+06,1.941764e+06,1.865781e+06
9,6.261989e+05,6.367300e+05,6.219488e+05,6.350228e+05,6.122824e+05,6.340312e+05,6.309823e+05,6.229593e+05,6.401217e+05,6.221589e+05


In [16]:
submissions['blend'] = (submissions.sum(axis=1))/len(submissions.columns)
sample_submission['price'] = submissions['blend'].values
sample_submission.to_csv('submission_blend_best.csv', index=False)
sample_submission.head(10)

,id,price
0,0,2.245929e+06
1,1,1.586638e+06
2,2,4.614771e+05
3,3,3.359788e+06
4,4,1.213223e+06
5,5,7.063356e+05
6,6,3.937116e+05
7,7,5.685914e+05
8,8,1.879716e+06
9,9,6.282436e+05


####  Итоговый результат MAPE = 11.24149